<a href="https://colab.research.google.com/github/ehsan-lari/Conformal-Prediction/blob/main/ConformalPrediction_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd

In [2]:
# Define CNN
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        return self.fc(self.conv(x))

In [3]:
# Data loading
transform = transforms.ToTensor()
dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
train_set, calib_set = random_split(dataset, [50000, 10000])
test_set = datasets.FashionMNIST(root="./data", train=False, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
calib_loader = DataLoader(calib_set, batch_size=64)
test_loader = DataLoader(test_set, batch_size=1000)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 209kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.89MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 14.0MB/s]


In [8]:
# Train model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FashionCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

model.train()
for epoch in range(10):  # Increase epochs for better accuracy
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        loss = loss_fn(model(X), y)
        loss.backward()
        optimizer.step()

In [9]:
# Calibrate using softmax scores
model.eval()
calib_scores = []
with torch.no_grad():
    for X, y in calib_loader:
        X = X.to(device)
        probs = F.softmax(model(X), dim=1).cpu().numpy()
        y = y.numpy()
        calib_scores += list(1 - probs[np.arange(len(y)), y])

calib_scores = np.array(calib_scores)
q = np.quantile(calib_scores, 0.95)  # 95% coverage

In [10]:
# Prediction sets
X_test, y_test = next(iter(test_loader))
X_test = X_test.to(device)
with torch.no_grad():
    probs = F.softmax(model(X_test), dim=1).cpu().numpy()

prediction_sets = [np.where(1 - p <= q)[0].tolist() for p in probs[:10]]

In [11]:
# Show first 10 results
df = pd.DataFrame({
    "True Label": y_test[:10].numpy(),
    "Prediction Set": prediction_sets
})

print(df)

   True Label Prediction Set
0           9            [9]
1           2            [2]
2           1            [1]
3           1            [1]
4           6            [6]
5           1            [1]
6           4            [4]
7           6            [6]
8           5            [5]
9           7            [7]
